## 1. Setup e Imports

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('✓ Imports concluídos')

## 2. Carregamento de Dados

In [ ]:
# Caminhos dos dados
SILVER_PATH = Path('../data/silver')
GOLD_PATH = Path('../data/gold')

# Verifica se os dados existem
print(f"Silver path exists: {SILVER_PATH.exists()}")
print(f"Gold path exists: {GOLD_PATH.exists()}")

In [ ]:
# Carrega dados da camada Silver (se existirem)
try:
    from deltalake import DeltaTable
    
    dt_onibus = DeltaTable(str(SILVER_PATH / 'onibus_posicoes'))
    df_onibus = dt_onibus.to_pandas()
    
    print(f"✓ Dados carregados: {len(df_onibus):,} registros")
    print(f"\nColunas: {df_onibus.columns.tolist()}")
    print(f"\nPrimeiras linhas:")
    display(df_onibus.head())
    
except Exception as e:
    print(f"⚠ Erro ao carregar dados: {e}")
    print("\nExecute o pipeline primeiro: python src/pipeline.py")
    df_onibus = None

## 3. Análise Exploratória

In [ ]:
if df_onibus is not None:
    # Estatísticas básicas
    print("=" * 60)
    print("ESTATÍSTICAS DESCRITIVAS")
    print("=" * 60)
    display(df_onibus.describe())
    
    # Informações do dataset
    print("\n" + "=" * 60)
    print("INFORMAÇÕES DO DATASET")
    print("=" * 60)
    df_onibus.info()

In [ ]:
if df_onibus is not None:
    # Valores ausentes
    print("=" * 60)
    print("VALORES AUSENTES")
    print("=" * 60)
    
    missing = df_onibus.isnull().sum()
    missing_pct = (missing / len(df_onibus) * 100).round(2)
    missing_df = pd.DataFrame({
        'Valores Ausentes': missing,
        'Percentual (%)': missing_pct
    })
    display(missing_df[missing_df['Valores Ausentes'] > 0])

## 4. Distribuição de Velocidade

In [ ]:
if df_onibus is not None and 'velocidade' in df_onibus.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma
    axes[0].hist(df_onibus['velocidade'].dropna(), bins=50, edgecolor='black')
    axes[0].set_title('Distribuição de Velocidade', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Velocidade (km/h)')
    axes[0].set_ylabel('Frequência')
    axes[0].axvline(df_onibus['velocidade'].mean(), color='red', 
                    linestyle='--', label=f'Média: {df_onibus["velocidade"].mean():.1f} km/h')
    axes[0].legend()
    
    # Boxplot
    axes[1].boxplot(df_onibus['velocidade'].dropna())
    axes[1].set_title('Boxplot de Velocidade', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Velocidade (km/h)')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas
    print("\nEstatísticas de Velocidade:")
    print(f"  Média: {df_onibus['velocidade'].mean():.2f} km/h")
    print(f"  Mediana: {df_onibus['velocidade'].median():.2f} km/h")
    print(f"  Desvio Padrão: {df_onibus['velocidade'].std():.2f} km/h")
    print(f"  Mínimo: {df_onibus['velocidade'].min():.2f} km/h")
    print(f"  Máximo: {df_onibus['velocidade'].max():.2f} km/h")

## 5. Análise Temporal

In [ ]:
if df_onibus is not None and 'hora' in df_onibus.columns:
    # Ônibus por hora
    onibus_por_hora = df_onibus.groupby('hora').size()
    
    plt.figure(figsize=(14, 6))
    onibus_por_hora.plot(kind='bar', color='steelblue', edgecolor='black')
    plt.title('Distribuição de Ônibus por Hora do Dia', fontsize=14, fontweight='bold')
    plt.xlabel('Hora do Dia')
    plt.ylabel('Número de Registros')
    plt.xticks(rotation=0)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
if df_onibus is not None and 'periodo_dia' in df_onibus.columns:
    # Velocidade por período do dia
    vel_periodo = df_onibus.groupby('periodo_dia')['velocidade'].agg(['mean', 'median', 'std'])
    
    print("\nVelocidade Média por Período do Dia:")
    display(vel_periodo.round(2))
    
    # Visualização
    plt.figure(figsize=(10, 6))
    vel_periodo['mean'].plot(kind='bar', color='coral', edgecolor='black')
    plt.title('Velocidade Média por Período do Dia', fontsize=14, fontweight='bold')
    plt.xlabel('Período')
    plt.ylabel('Velocidade Média (km/h)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. Análise Geográfica

In [ ]:
if df_onibus is not None and all(col in df_onibus.columns for col in ['latitude', 'longitude']):
    # Scatter plot de posições
    plt.figure(figsize=(12, 10))
    
    # Amostra para melhor visualização
    sample_size = min(10000, len(df_onibus))
    df_sample = df_onibus.sample(n=sample_size, random_state=42)
    
    scatter = plt.scatter(
        df_sample['longitude'],
        df_sample['latitude'],
        c=df_sample['velocidade'] if 'velocidade' in df_sample.columns else None,
        cmap='RdYlGn',
        alpha=0.6,
        s=10
    )
    
    plt.colorbar(scatter, label='Velocidade (km/h)')
    plt.title(f'Posições dos Ônibus em BH (amostra de {sample_size:,} registros)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Estatísticas geográficas
    print("\nLimites Geográficos:")
    print(f"  Latitude:  {df_onibus['latitude'].min():.6f} a {df_onibus['latitude'].max():.6f}")
    print(f"  Longitude: {df_onibus['longitude'].min():.6f} a {df_onibus['longitude'].max():.6f}")

## 7. Score de Qualidade

In [ ]:
if df_onibus is not None and '_quality_score' in df_onibus.columns:
    # Distribuição do score de qualidade
    plt.figure(figsize=(10, 6))
    plt.hist(df_onibus['_quality_score'], bins=30, edgecolor='black', color='lightgreen')
    plt.title('Distribuição do Score de Qualidade dos Dados', fontsize=14, fontweight='bold')
    plt.xlabel('Score de Qualidade (0-1)')
    plt.ylabel('Frequência')
    plt.axvline(df_onibus['_quality_score'].mean(), color='red', 
                linestyle='--', label=f'Média: {df_onibus["_quality_score"].mean():.3f}')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nScore Médio de Qualidade: {df_onibus['_quality_score'].mean():.3f}")
    print(f"Score Mínimo: {df_onibus['_quality_score'].min():.3f}")
    print(f"Score Máximo: {df_onibus['_quality_score'].max():.3f}")

## 8. Conclusões

Este notebook apresentou uma análise exploratória dos dados de mobilidade urbana de Belo Horizonte:

- **Distribuição de Velocidade**: Analisamos a velocidade dos ônibus
- **Padrões Temporais**: Identificamos horários de pico
- **Cobertura Geográfica**: Visualizamos a distribuição espacial
- **Qualidade dos Dados**: Avaliamos a confiabilidade das informações

Para análises mais avançadas, veja o notebook `02_analise_metricas.ipynb`.